Using dot_connect's private key utilities to support key pair authentication and rotation.


In [ ]:
import dot_connect
import dot_connect.utils

Generate a new private key pair and write them to local files (for now).


In [ ]:
password = dot_connect.utils.generate_random_password()
priv, pub = dot_connect.utils.generate_key_pair(password)

In [ ]:
client = dot_connect.aws.connect("secretsmanager")

In [ ]:
dot_connect.utils.create_aws_secret(client, "sfpscogs-twhite-priv", priv)
dot_connect.utils.create_aws_secret(client, "sfpscogs-twhite-password", password)

In [ ]:
con = dot_connect.snowflake.connect()
cursor = con.cursor()

cursor.execute("USE ROLE USERADMIN;").fetchall()

cursor.execute(
    f"""
CREATE OR REPLACE USER KEY_USER WITH RSA_PUBLIC_KEY = '{dot_connect.utils.strip_public_key_headers(pub)}';
"""
).fetchall()

con.close()

Connect to Snowflake using our new user and the key pair.


In [ ]:
import snowflake.connector
import os

pkb = dot_connect.utils.serialize_key(
    client.get_secret_value(SecretId="sfpscogs-twhite-priv").get("SecretBinary"),
    client.get_secret_value(SecretId="sfpscogs-twhite-password").get("SecretBinary"),
)

con = snowflake.connector.connect(
    **dict(
        account=os.environ.get("SNOWFLAKE_ACCOUNT"), user="KEY_USER", private_key=pkb
    )
)

In [ ]:
print(con.cursor().execute("SELECT 'I AM ' || CURRENT_USER();").fetchall())
con.close()